# Airflow DAG Generation - Model Inference on GPU

This notebook runs inference on the test dataset comparing:
- **Base Model**: Qwen/Qwen2.5-Coder-1.5B-Instruct
- **Fine-tuned Model**: andrea-t94/qwen2.5-1.5b-airflow-instruct (LoRA adapter)

**Dataset**: andrea-t94/airflow-dag-dataset (test split)

## Setup Instructions
1. Runtime → Change runtime type → T4 GPU (or better)
2. Run all cells in order
3. Models and dataset will be cached for re-runs

## 1. GPU Check and Environment Setup

In [1]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected. Please enable GPU in Runtime → Change runtime type")

Wed Dec 17 10:39:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Install Dependencies (Unsloth for Fast Inference)

In [10]:
# Install Unsloth for optimized inference with flash attention
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q datasets>=3.2.0

print("✅ Unsloth and dependencies installed successfully")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.6/180.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 123.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 24.0 MB/s eta 0:00:00
✅ Unsloth and dependencies installed successfully


## 3. Load Test Dataset from HuggingFace

In [3]:
from datasets import load_dataset
import json

# Load test split (will be cached automatically)
print("Loading test dataset from HuggingFace...")
dataset = load_dataset(
    "andrea-t94/airflow-dag-dataset",
    split="test",
    download_mode="reuse_cache_if_exists"  # Use cached version if available
)

print(f"\n✅ Loaded {len(dataset)} test examples")

# Show dataset statistics
airflow_count = sum(1 for x in dataset if x.get('source') == 'airflow')
magpie_count = sum(1 for x in dataset if x.get('source') == 'magpie')

print(f"\nDataset composition:")
print(f"  - Airflow examples: {airflow_count} ({airflow_count/len(dataset)*100:.1f}%)")
print(f"  - Magpie examples: {magpie_count} ({magpie_count/len(dataset)*100:.1f}%)")

# Preview first example
print(f"\nFirst example:")
print(json.dumps(dataset[0]['messages'][:2], indent=2))  # Show system + user message

Loading test dataset from HuggingFace...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/560k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/7414 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/412 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/412 [00:00<?, ? examples/s]


✅ Loaded 412 test examples

Dataset composition:
  - Airflow examples: 342 (83.0%)
  - Magpie examples: 70 (17.0%)

First example:
[
  {
    "role": "system",
    "content": "You are an expert Apache Airflow developer. Generate complete, valid Airflow DAGs based on given requirements."
  },
  {
    "role": "user",
    "content": "Create a data pipeline that demonstrates loading sample product data into a Snowflake table and validating the data load. The pipeline should insert 12 product records and then verify the total number of rows matches the expected count.\n\nAirflow Version: 3.0.1"
  }
]


: 

In [8]:
dataset[0]

{'messages': [{'role': 'system',
   'content': 'You are an expert Apache Airflow developer. Generate complete, valid Airflow DAGs based on given requirements.'},
  {'role': 'user',
   'content': 'Create a data pipeline that demonstrates loading sample product data into a Snowflake table and validating the data load. The pipeline should insert 12 product records and then verify the total number of rows matches the expected count.\n\nAirflow Version: 3.0.1'},
  {'role': 'assistant',
   'content': '"""\nExample use of Snowflake Snowpark Python related operators.\n"""\n\nfrom __future__ import annotations\n\nfrom datetime import datetime\nfrom typing import TYPE_CHECKING\n\nif TYPE_CHECKING:\n    from snowflake.snowpark import Session\n\nfrom airflow import DAG\nfrom airflow.providers.snowflake.operators.snowpark import SnowparkOperator\n\nSNOWFLAKE_CONN_ID = "my_snowflake_conn"\nDAG_ID = "example_snowpark_operator"\n\nwith DAG(\n    DAG_ID,\n    start_date=datetime(2024, 1, 1),\n    sched

## 4. Load Models

We'll load both:
1. Base model: Qwen/Qwen2.5-Coder-1.5B-Instruct
2. Fine-tuned model: Base + LoRA adapter from andrea-t94/qwen2.5-1.5b-airflow-instruct

In [11]:
from unsloth import FastLanguageModel
import torch

# Model identifiers
BASE_MODEL_ID = "Qwen/Qwen2.5-Coder-1.5B-Instruct"
FINETUNED_ADAPTER_ID = "andrea-t94/qwen2.5-1.5b-airflow-instruct"

# Unsloth configuration
max_seq_length = 4096
dtype = None            # Auto-detect (float16 for T4)
load_in_4bit = False    # False for speed/precision, True for memory savings

print("Loading models with Unsloth...")
print("="*60)

# ---------------------------------------------------------
# 1. Loading the BASE model (Pure Base)
# ---------------------------------------------------------
print("\n1. Loading BASE model...")
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_ID,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(base_model)
print(f"✅ Base model loaded: {BASE_MODEL_ID}")

# ---------------------------------------------------------
# 2. Loading the FINE-TUNED model (Base + Adapter)
# ---------------------------------------------------------
print("\n2. Loading FINE-TUNED model...")
# OPTION A: The Unsloth "Magic" Way 
# (Pass the Adapter ID directly; Unsloth finds the base model automatically)
finetuned_model, _ = FastLanguageModel.from_pretrained(
    model_name = FINETUNED_ADAPTER_ID, # <--- Put Adapter ID here
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Enable inference mode
FastLanguageModel.for_inference(finetuned_model)
print(f"✅ Fine-tuned model loaded: {FINETUNED_ADAPTER_ID}")

print("\n" + "="*60)
print("✅ All models loaded.")

/tmp/ipython-input-444238182.py:1: UserWarning: WARNING: Unsloth should be imported before [transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading models with Unsloth...

1. Loading BASE model...
==((====))==  Unsloth 2025.12.5: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.12.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


✅ Fine-tuned model loaded: andrea-t94/qwen2.5-1.5b-airflow-instruct

✅ All models loaded.


## 5. Inference Configuration

In [12]:
# Unsloth TextStreamer configuration for fast generation
from unsloth.chat_templates import get_chat_template

# Apply chat template to tokenizer
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",
)

# Generation parameters optimized for Unsloth
GENERATION_CONFIG = {
    "max_new_tokens": 4096,  # Reduced for speed
    "temperature": 0.1,
    "top_p": 0.9,
    "do_sample": True,
    "use_cache": True,  # Enable KV cache for speed
}

print("Inference configuration (Unsloth optimized):")
for key, value in GENERATION_CONFIG.items():
    print(f"  {key}: {value}")

Inference configuration (Unsloth optimized):
  max_new_tokens: 4096
  temperature: 0.1
  top_p: 0.9
  do_sample: True
  use_cache: True


## 6. Helper Functions

In [ ]:
def extract_code_from_response(response_text):
    """Extract Python code from model response."""
    # Try to find code block
    if "```python" in response_text:
        start_idx = response_text.find("```python") + len("```python")
        end_idx = response_text.find("```", start_idx)
        if end_idx != -1:
            return response_text[start_idx:end_idx].strip()
    
    # Try generic code block
    if "```" in response_text:
        parts = response_text.split("```")
        if len(parts) >= 3:
            return parts[1].strip()
    
    return response_text.strip()

✅ Helper functions defined (Unsloth optimized)


## 7. Run Inference on Test Dataset (Optimized)

This will process models **separately** to maximize speed and avoid memory pressure.

In [19]:
import time
from tqdm.auto import tqdm
import gc
import torch
from unsloth import FastLanguageModel

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
MAX_EXAMPLES = 10  # Set to None for full run
# Increase Batch Size for parallel processing 
# (For a 1.5B model on T4 GPU, you can likely push this to 16 or 32)
BATCH_SIZE = 8     

# Create the test subset
test_examples = dataset.select(range(min(MAX_EXAMPLES, len(dataset)))) if MAX_EXAMPLES else dataset

# Helper to slice dataset cleanly
def get_batch_as_list(dataset, start_idx, batch_size):
    end_idx = min(start_idx + batch_size, len(dataset))
    return dataset.select(range(start_idx, end_idx)).to_list()

# Helper to extract code (ensure this is defined)
def extract_code_from_response(text):
    if "```python" in text:
        return text.split("```python")[1].split("```")[0].strip()
    if "```" in text:
        return text.split("```")[1].strip()
    return text.strip()

# ---------------------------------------------------------
# PARALLEL GENERATION FUNCTION (The Speed Upgrade)
# ---------------------------------------------------------
def generate_parallel(model, tokenizer, examples, model_name="model"):
    results = []
    
    # 1. Prepare Prompts
    prompts = []
    for example in examples:
        messages = example['messages']
        # Apply template to inputs only
        prompt = tokenizer.apply_chat_template(
            messages[:-1], 
            tokenize=False, 
            add_generation_prompt=True
        )
        prompts.append(prompt)
    
    # 2. Batch Tokenize (Left Padding is crucial here!)
    # We do this inside the function to ensure the tokenizer config is respected
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,           # Pad to the longest sequence in this batch
        truncation=True,
        max_length=4096,
    ).to("cuda")

    # 3. Generate (Processes all 8 rows simultaneously)
    outputs = model.generate(
        **inputs,
        **GENERATION_CONFIG,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    # 4. Batch Decode
    # Slice off the input prompt tokens from the output
    generated_ids = outputs[:, inputs['input_ids'].shape[1]:]
    decoded_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    # 5. Reassemble Results
    for i, text in enumerate(decoded_texts):
        original_msg = examples[i]['messages']
        results.append({
            'messages': original_msg[:-1] + [{'role': 'assistant', 'content': extract_code_from_response(text)}],
            'metadata': {
                **(examples[i].get('metadata') or {}),
                'model': model_name,
                'inference_time': time.time()
            }
        })
        
    return results

# ==========================================
# MAIN EXECUTION
# ==========================================

print(f"Strategy: True Parallel Batching (Batch Size: {BATCH_SIZE})")

# --- STEP 1: BASE MODEL ---
print("\n🤖 STEP 1/2: Loading & Running BASE model...")
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_ID,
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=False,
)
FastLanguageModel.for_inference(base_model)

# CRITICAL: Configure Tokenizer for Batch Generation
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

base_results = []
start_time = time.time()

for i in tqdm(range(0, len(test_examples), BATCH_SIZE), desc="Base Model"):
    batch_data = get_batch_as_list(test_examples, i, BATCH_SIZE)
    batch_results = generate_parallel(base_model, tokenizer, batch_data, "Qwen2.5-Base")
    base_results.extend(batch_results)

# Cleanup
del base_model
torch.cuda.empty_cache()
gc.collect()

# --- STEP 2: FINETUNED MODEL ---
print("\n✨ STEP 2/2: Loading & Running FINETUNED model...")
finetuned_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=FINETUNED_ADAPTER_ID,
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=False,
)
FastLanguageModel.for_inference(finetuned_model)

# CRITICAL: Re-configure Tokenizer (Loading a new model might reset it)
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

finetuned_results = []

for i in tqdm(range(0, len(test_examples), BATCH_SIZE), desc="Finetuned Model"):
    batch_data = get_batch_as_list(test_examples, i, BATCH_SIZE)
    batch_results = generate_parallel(finetuned_model, tokenizer, batch_data, "Qwen2.5-Finetuned")
    finetuned_results.extend(batch_results)

print("\n" + "="*60)
print(f"✅ Inference Complete.")
print(f"Total Base Results: {len(base_results)}")
print(f"Total Finetuned Results: {len(finetuned_results)}")
print("="*60)

Strategy: True Parallel Batching (Batch Size: 8)

🤖 STEP 1/2: Loading & Running BASE model...
==((====))==  Unsloth 2025.12.5: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Base Model:   0%|          | 0/2 [00:00<?, ?it/s]


✨ STEP 2/2: Loading & Running FINETUNED model...
==((====))==  Unsloth 2025.12.5: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Finetuned Model:   0%|          | 0/2 [00:00<?, ?it/s]


✅ Inference Complete.
Total Base Results: 10
Total Finetuned Results: 10


In [21]:
finetuned_results

[{'messages': [{'role': 'system',
    'content': 'You are an expert Apache Airflow developer. Generate complete, valid Airflow DAGs based on given requirements.'},
   {'role': 'user',
    'content': 'Create a data pipeline that demonstrates loading sample product data into a Snowflake table and validating the data load. The pipeline should insert 12 product records and then verify the total number of rows matches the expected count.\n\nAirflow Version: 3.0.1'},
   {'role': 'assistant',
    'content': 'from __future__ import annotations\n\nimport os\nfrom datetime import datetime\n\nfrom airflow import DAG\nfrom airflow.providers.snowflake.operators.snowflake import SnowflakeOperator\nfrom airflow.utils.trigger_rule import TriggerRule\n\nENV_ID = os.environ.get("SYSTEM_TESTS_ENV_ID")\nSNOWFLAKE_SAMPLE_TABLE = "sample"\nSNOWFLAKE_CONN_ID = "snowflake"\n\nwith DAG(\n    dag_id=f"example_snowflake_insert_many_{ENV_ID}",\n    start_date=datetime(2021, 1, 1),\n    schedule=None,\n    catchup

In [35]:
final_consolidated_results = []

# Iterate through the original dataset and our two result lists simultaneously
for i, example in enumerate(test_examples):
    
    # 1. Extract inputs (System + User)
    input_messages = example['messages'][:-1]
    
    # 2. Extract Original "Gold" Output (The last message in the dataset)
    original_target = example['messages'][-1]['content']
    
    # 3. Extract Generated Outputs (FIX IS HERE)
    # We look into the 'messages' list and grab the last item (the Assistant's reply)
    base_code = base_results[i]['messages'][-1]['content']
    finetuned_code = finetuned_results[i]['messages'][-1]['content']
    
    # 4. Build the Combined Object
    combined_entry = {
        'id': i,
        'messages': input_messages,  # The prompt context
        'responses': {
            'original': extract_code_from_response(original_target),  # Truth
            'base_model': base_code,                                  # Just the code string
            'finetuned': finetuned_code                               # Just the code string
        },
        'metadata': example.get('metadata', {})
    }
    
    final_consolidated_results.append(combined_entry)

# --- VERIFICATION ---
print("\nSample Result Structure (Corrected):")
print(f"Instruction: {final_consolidated_results[0]['messages'][1]['content'][:50]}...")
print("-" * 30)
print(f"Original (First 50 chars):  {final_consolidated_results[0]['responses']['original'][:50]}...")
print(f"Base (First 50 chars):      {final_consolidated_results[0]['responses']['base_model'][:50]}...")
print(f"Finetuned (First 50 chars): {final_consolidated_results[0]['responses']['finetuned'][:50]}...")

print(f"\n✅ Successfully combined {len(final_consolidated_results)} examples.")


Sample Result Structure (Corrected):
Instruction: Create a data pipeline that demonstrates loading s...
------------------------------
Original (First 50 chars):  """
Example use of Snowflake Snowpark Python relat...
Base (First 50 chars):      from airflow import DAG
from airflow.providers.sno...
Finetuned (First 50 chars): from __future__ import annotations

import os
from...

✅ Successfully combined 10 examples.


In [36]:
final_consolidated_results

[{'id': 0,
  'messages': [{'role': 'system',
    'content': 'You are an expert Apache Airflow developer. Generate complete, valid Airflow DAGs based on given requirements.'},
   {'role': 'user',
    'content': 'Create a data pipeline that demonstrates loading sample product data into a Snowflake table and validating the data load. The pipeline should insert 12 product records and then verify the total number of rows matches the expected count.\n\nAirflow Version: 3.0.1'}],
  'responses': {'original': '"""\nExample use of Snowflake Snowpark Python related operators.\n"""\n\nfrom __future__ import annotations\n\nfrom datetime import datetime\nfrom typing import TYPE_CHECKING\n\nif TYPE_CHECKING:\n    from snowflake.snowpark import Session\n\nfrom airflow import DAG\nfrom airflow.providers.snowflake.operators.snowpark import SnowparkOperator\n\nSNOWFLAKE_CONN_ID = "my_snowflake_conn"\nDAG_ID = "example_snowpark_operator"\n\nwith DAG(\n    DAG_ID,\n    start_date=datetime(2024, 1, 1),\n   

## 8. Save Results on drive (optional)

In [ ]:
import json
from datetime import datetime

# 1. Create a timestamped filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"airflow_model_comparison_{timestamp}.jsonl"

# 2. Save the 'final_consolidated_results' list
# (This assumes you ran the combining loop from the previous step)
print(f"Saving {len(final_consolidated_results)} examples to {output_file}...")

with open(output_file, 'w') as f:
    for entry in final_consolidated_results:
        # verify we are saving the clean structure
        f.write(json.dumps(entry) + '\n')

print(f"✅ Comparison data saved successfully.")


✅ Results saved:
   Base model: generated_dags_base_model_20251217_114149.jsonl
   Fine-tuned model: generated_dags_finetuned_model_20251217_114149.jsonl



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Downloads initiated


## 11. Cleanup (Optional)

Free up GPU memory if needed.

In [ ]:
import gc

# Delete models to free memory
del base_model
del finetuned_model
torch.cuda.empty_cache()
gc.collect()

print("✅ GPU memory cleared")
!nvidia-smi